# REST-API
  - Raid API 사용해서 실습하기 

Skate ipsum - Endpoint 실습 

In [1]:
import requests

url = "https://mashape-community-skate-ipsum.p.rapidapi.com/3/1/JSON"

headers = {
    'x-rapidapi-host': "mashape-community-skate-ipsum.p.rapidapi.com",
    'x-rapidapi-key': "3788e2c4cdmshe08626798a61bc3p12c7a0jsnf3abb526cfee"
    }

response = requests.request("GET", url, headers=headers)

print(response.text)

["Skate ipsum dolor sit amet,  wax vert nose freestyle casper Tod Swank. Nose slide NoMeansNo spine darkslide Tracker birdie. Tony Magnusson blunt nollie ho-ho Saran Wrap crooked grind. Salad grind poseur flail Caballerial yeah lipslide. Hang up hurricane ollie north rail slide darkslide. Speed wobbles 540 pivot aerial ollie north. Sponsored tic-tac heel flip trucks feeble. Betty John Grigley fastplant regular footed nose-bump dude. Kick-nose rock and roll Upland backside concave carve. Disaster hospital flip grab concave full-cab. Crail slide impossible sketchy kingpin tuna-flip. Grab sketchy gap rail fakie out. ","Hang ten pivot opposite footed downhill Blind hurricane transition. Blunt cab flip full pipe steps nosepicker deck. Griptape casper slide Primo slide wax 180 slam. Nosegrind nose slide kickflip hand rail bruised heel alley oop Gullwing. Mute-air nosebone shoveit ollie hole cab flip front foot impossible. Ollie north tail blunt Paul Rodriguez 720 pressure flip boned out. Han

## Weather Forecast - Endpoint 활용 방법 
  - 날씨를 통해 여행지 선정하기 

Step 1. Import weather data into a Python program
  - Weather Forecast API 사용 

In [9]:
import requests
import re

cities = ['London,uk', 'Paris,fr', 'Porto,pt']
weather_dict = {}
def city_forecast(city):
  url = "https://community-open-weather-map.p.rapidapi.com/forecast"
  querystring = {"q": city}
  headers = {
      'x-rapidapi-host': "community-open-weather-map.p.rapidapi.com",
      'x-rapidapi-key': "3788e2c4cdmshe08626798a61bc3p12c7a0jsnf3abb526cfee"
      }

  response = requests.request("GET", url, headers=headers, params=querystring)
  return response.json()

In [10]:
for city in cities:
  weather_dict[city] = city_forecast(city)

Step 2. Prepare data for estimation

In [11]:
def get_day_weather(pred):
  pattern =re.compile(" [0-9]*:[0-9]*:[0-9]*")
  t = pattern.search(pred['dt_txt'])

  hour = int(t[0].split(':')[0])
  
  if 10<=hour<=19:
        return True

  return False

day_weather = {}

# USe filter function select the predictions of only the day temperature
for city in weather_dict.keys():
  day_weather[city] = list(filter(get_day_weather, weather_dict[city]['list']))

Step 3. Estimate the best city for travel

In [12]:
def travel_estimator(weather_stat):
  estimation = {}

  for city in weather_stat.keys():    
        estimation[city] = {}
        estimation[city]['clear_sky_count'] = 0
        estimation[city]['av_temp'] = 0

        for prediction in day_weather[city]:
          estimation[city]['av_temp'] += prediction['main']['temp']
          if prediction['weather'][0]['description'] in ['clear sky', 'few clouds']:
            estimation[city]['clear_sky_count'] += 1          

        estimation[city]['av_temp'] = round(estimation[city]['av_temp'] / len(day_weather[city]), 2)
        # convert temperature to celsius
        estimation[city]['av_temp_cels'] = round(estimation[city]['av_temp'] - 273.15, 2)

  return estimation

travel_rating = travel_estimator(day_weather)

# sort cities by clear sky forecasts
sorted_travel_rating = sorted(travel_rating.items() ,  key=lambda x: x[1]['av_temp_cels'] )


for city in sorted_travel_rating:
  print(city[0])
  print('Clear sky forecasts:', city[1]['clear_sky_count'])  
  print('Average temperature:', city[1]['av_temp_cels'], "C")
  print('n')

London,uk
Clear sky forecasts: 2
Average temperature: 14.8 C
n
Paris,fr
Clear sky forecasts: 1
Average temperature: 16.76 C
n
Porto,pt
Clear sky forecasts: 1
Average temperature: 19.13 C
n
